In [13]:
import sympy as sp
import numpy as np
thp, thd, tp, td, lp, ld, alp, r, s = sp.symbols(r'\theta_p \theta_d t_p t_d l_p l_d \alpha r s', real = True)
kda = td / (ld * r)
kd1 = alp * kda
kd = 2 * (kda - kd1) * s / ld + kd1
display(kd)

\alpha*t_d/(l_d*r) + s*(-2*\alpha*t_d/(l_d*r) + 2*t_d/(l_d*r))/l_d

In [14]:
with sp.assuming(sp.Q.nonzero(ld)):
    dp = sp.integrate(r * kd, (s, 0, lp))
display(sp.simplify(dp))

l_p*t_d*(\alpha*l_d - l_p*(\alpha - 1))/l_d**2

프록시말텐던 중립상태에서 디스털만 구동 -- 프록시말 부분에서는 유효곡률은 그냥 곡률의 1/2 (r -> 2r)이 되는 효과

아니면 컴프레션 무시하는것도 좋을듯 그냥 dp만큼 줄고 늘었으니 dp만큼 카운터치면 되는거아님? >> 이러면 프록시말 길이변동은 없음

In [15]:
dpnew = dp / 2

In [ ]:
# 회전행렬 테스트
mat = np.array([1,0])
theta = np.pi / 2
rot = np.array([[np.cos(theta), -np.sin(theta)],
                [np.sin(theta), np.cos(theta)]])
rotval = rot @ mat
display(rotval)